# Parcels sample file

As part of this Notebook, we will use a simple *[Parcels](https://oceanparcels.org/)* output file to highlight the required steps to preprocess and dataset into a format that can be ingest by the *CloudDrift* library.

## Dataformat module

The `dataformat.py` module contains the class `create_ragged_array` to transform a series of archives into an *Awkward Array* where all variables is stored as a *ragged array*. The module also contains `read_from_netcdf` and `read_from_parquet` to initialize the *Awkward Array* directly from an previously preprocessed archive. Right now, it *only* supports local array but we will soon add the possibility of *lazy-loading* array stored in the Cloud.

In [1]:
import sys
import numpy as np

In [2]:
sys.path.insert(0, '../')
from clouddrift import dataformat

The main class of this module is *create_ragged_array* and is used to create a single archive that can be saved to a netCDF or Parquet file. The signature of the class is:

In [3]:
dataformat.create_ragged_array?

Init signature:
dataformat.create_ragged_array(
    indices: list,
    preprocess_func: collections.abc.Callable[[int], xarray.core.dataset.Dataset],
    vars_coords: dict,
    vars_meta: list = [],
    vars_data: list = [],
    rowsize_func: collections.abc.Callable[[int], int] = None,
)
Docstring:      <no docstring>
File:           ~/Library/CloudStorage/OneDrive-FloridaStateUniversity/projects/clouddrift/clouddrift/dataformat.py
Type:           type
Subclasses:     


## Dataset-specific functions

Since each dataset is different, we have to create specific functions to preprocess the dataset (`preprocess_func`) and return the metadata and data of a single trajectory. This was inspired by the [Pangeo Forge](https://pangeo-forge.readthedocs.io/en/latest/) project. The class *create_ragged_array* will use those functions to create the single archive of ragged arrays. More precisely, it requires:
- a list of indices (or identification number) that will be concatenate into the ragged array format
- a preprecessing function with the following signature:
    - `Signature: preprocess_func(index: int) -> xarray.core.dataset.Dataset`, where the index parameter is an identifier of a trajectory, e.g. the identification number of an Argo float) and returns an *xarray Dataset*. 
- a dictionnary mapping the mandatory coordinates list to the name of those variables in the dataset, e.g.
    coords = {'ids': 'number', 'time': 't', 'longitude': 'lon', 'latitude': 'lat'}
- an optional list of variable names containing metadata information about the trajectory (size: 1 per trajectory)
- an optional list of variable names containing the data along the trajectory (size: number of observations per trajectory)
- an optional funcition that returns directly the number of observation of a trajectory (`Signature: rowsize_func(index: int) -> int`)
    
This function can performs all type of operations, such as formatting the date, changing the type of variables, modifying the metadata, etc. We provide preprocessing function for different datasets in the `data/recipes/` folder. The class also needs to *initially* calculate the sum of all observations. By default, this is performed using an lambda function: `lambda i: self.preprocess_func(i).dims['obs']`. To *speed up* this process, in the situation where a lot of preprocessing are performed, it is possible to provide a function `rowsize_func`, that returns directly the number of observation of a trajectory (`Signature: rowsize_func(index: int) -> int`)

Because there is only one `dat` file, it is automatically loaded when importing the module.

In [4]:
%load_ext autoreload
%autoreload 2
from data import parcels

Dataset example.nc already in '../data/raw/parcels/'.


We can test the preprocessing function by calling it with one of the identification numbers.

In [5]:
parcels.preprocess(1000)

<xarray.Dataset>
Dimensions:     (traj: 1, obs: 217)
Dimensions without coordinates: traj, obs
Data variables:
    trajectory  (traj, obs) float64 ...
    time        (traj, obs) datetime64[ns] ...
    lat         (traj, obs) float64 ...
    lon         (traj, obs) float64 ...
    z           (traj, obs) float64 ...
    ID          (traj) int64 2984
    rowsize     (traj) int64 217
    ids         (obs) int64 2984 2984 2984 2984 2984 ... 2984 2984 2984 2984
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        2.3.0
    parcels_mesh:           spherical

It is now possible to create the ragged array and either save a netCDF, parquet file, or simply output an Awkward Array that can be used for analysis.

In [6]:
indices = np.arange(0, 1984)
coords = {'ids': 'ids', 'time': 'time', 'lon': 'lon', 'lat': 'lat'}
metadata = ['ID', 'rowsize']

ra = dataformat.create_ragged_array(indices,
                        parcels.preprocess,
                        coords, 
                        metadata, 
                        )

Filling the ragged array: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1984/1984 [00:02<00:00, 672.85it/s]


## Export

In [7]:
ra.to_parquet('../data/process/parcels.parquet')

In [8]:
ak = ra.to_awkward()

In [9]:
ak.ID

<Array [1984, 1985, 1986, ..., 3966, 3967] type='1984 * int64[parameters={"...'>

In [10]:
ak.fields

['ID', 'rowsize', 'obs']

In [11]:
ak.obs.fields

['ids', 'time', 'lon', 'lat']

## Read

In [12]:
ak2 = dataformat.read_from_parquet('../data/process/parcels.parquet')

In [13]:
ak2.ID

<Array [1984, 1985, 1986, ..., 3966, 3967] type='1984 * int64[parameters={"...'>